In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import logging
import tensorflow as tf
import tempfile
import os
from sklearn.metrics import accuracy_score

logging.getLogger('tensorflow').setLevel(logging.ERROR)
tf.autograph.set_verbosity(1)
np.set_printoptions(suppress=True)

tmpdir = tempfile.mkdtemp()

In [ ]:
cp_train = pd.read_csv(
    "/home/werner/Projects/tensor_flow/tensor_data_p0_0002.csv",
  header=[0])

cp_train.head()

cp_features = cp_train.copy()
list_to_pop = ['angleX','angleY','angleZ']
cp_labelsX = np.array(cp_features.pop('angleX'))
cp_labelsY = np.array(cp_features.pop('angleY'))
cp_labelsZ = np.array(cp_features.pop('angleZ'))

cp_labels = np.array(cp_labelsX)
cp_labels = np.column_stack((cp_labels, cp_labelsY))
cp_labels = np.column_stack((cp_labels, cp_labelsZ))
cp_features = np.array(cp_features)


print(cp_labels.shape, cp_labels.dtype)
print(cp_features.shape, cp_features.dtype)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8,)),
    tf.keras.layers.Normalization(),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=3,)
                             ])

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.0001))

history = model.fit(cp_features, cp_labels, epochs=200, verbose=1)

print("Finished training the model")
plt.plot(history.history['loss'])
print(history.history['loss'][-1])



In [ ]:
# print(tf.__version__)

In [ ]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size/1024,3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size/(1024*1024),3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + 'bytes')

In [ ]:
Keras_Model_name = "tf_model.h5"
model.save(Keras_Model_name)
convert_bytes(get_file_size(Keras_Model_name), "MB")


In [ ]:
tf_lite_model_file_name = "angle_model.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_model = tf_lite_converter.convert()
name = tf_lite_model_file_name
open(name, 'wb').write(tf_lite_model)
convert_bytes(get_file_size(tf_lite_model_file_name), 'KB')


In [ ]:
interpreter = tf.lite.Interpreter(model_path=name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Input shape: ', input_details[0]['shape'])
print('Input type: ', input_details[0]['dtype'])
print('Output shape: ', output_details[0]['shape'])
print('Output type: ', output_details[0]['dtype'])

In [ ]:
# test_data = cp_labels

# interpreter.set_tensor(input_details[0]['index'], test_data)
# interpreter.invoke()
# test_ans = interpreter.get_tensor(output_details[0]['index'])
# prediction_classes = np.argmax(test_ans, axis=1)
# acc = accuracy_score(prediction_classes, cp_features)


In [ ]:
# model.evaluate(cp_features, cp_labels)
# module_no_signatures_path = os.path.join(tmpdir, 'module_no_signatures')
# module(tf.constant(0.))
# print('Saving model...')
# tf.saved_model.save(module, module_no_signatures_path)